### **멀티모달 프롬프트 전송**

* *이어서 진행*
* *befor: 로컬 이미지 전송*

  ```
  ↓
  ```

#### **Google Cloud Storage (`GCS`)** 에서 문서 보내기

* 구글 클라우드 콘솔 실습에서 했던 내용
  * 예시 문서는 Google과 토론토 대학교의 연구진이 작성한 논문인 ["Attention is All You Need"](https://arxiv.org/abs/1706.03762)
  * `Gemini`를 활용한 문서 이해의 더 많은 예시를 보려면 다음 노트북을 확인할 것
  * [Gemini를 활용한 문서 처리](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/document_processing.ipynb)

---

* 로컬 환경에서 `GCS`에 접근하기 위한 **권한 문제**

  * 구글 클라우드 콘솔에서는 이미 사용자의 계정 정보가 인증되어 있기 때문에 별도의 절차 없이 GCS에 파일을 업로드하거나 접근할 수 있음
  * 하지만 로컬 컴퓨터에서는 구글 클라우드 환경과 분리되어 있어, GCS에 접근하기 위한 **인증 키나 권한 설정(Credential)**이 반드시 필요

---

* 로컬에서 `GCS`에 파일 업로드하는 방법

  * **서비스 계정 키(Service Account Key)**: 
    * 로컬 개발 환경에서 가장 일반적인 방법
    * 구글 클라우드에서 서비스 계정을 생성하고 키 파일을 다운로드하여 환경 변수로 설정하면 해당 계정의 권한으로 `GCS`에 접근 가능
  * **개인 사용자 인증(User Authentication)**: 
    * `gcloud auth login` 명령어를 사용해 개인 계정으로 로그인하는 방법
    * 로컬에서 실행되는 코드가 사용자의 계정 권한을 임시로 빌려 `GCS`에 접근 가능

---

* *재현하지 않고 다음으로 넘어감*

---

### 기본 설정

In [ ]:
# 로컬에서 실행 시
from IPython.display import Markdown, display, HTML
from google import genai
from google.genai import types

import os
from dotenv import load_dotenv


# .env 파일 로드
load_dotenv()   

# 클라이언트 초기화
# 단일 클라이언트 객체 생성하기
# API 키는 GEMINI_API_KEY 환경 변수에서 자동으로 로드

client = genai.Client()

# 1. 사용할 모델 지정 (gemini-2.5-flash-lite)
# 2. '헬로우' 프롬프트로 콘텐츠 생성 요청
try:
    response = client.models.generate_content(
        model='gemini-2.5-flash-lite',                     # 사용할 모델 지정 (gemini-2.5-flash-lite)
        contents='hello'                                   # 요청하신 프롬프트 '헬로우'
    )

    # 응답 텍스트 출력
    print("\n--- 모델 응답 텍스트 ---")
    print(response.text)

    # 응답 전체 내용(JSON 형식) 출력 (디버깅이나 상세 정보 확인용)
    print("\n--- 모델 응답 전체 JSON ---")
    print(response.model_dump_json(
        exclude_none=True, indent=4))

except Exception as e:
    print(f"\n모델 호출 중 오류가 발생했습니다: {e}")
    print("다음 사항들을 확인해주세요:")
    print("1. 인터넷 연결 상태")
    print("2. GEMINI_API_KEY 환경 변수가 올바르고 유효한지")
    print("3. Google Cloud 프로젝트에서 Gemini API가 활성화되어 있는지")
    print("4. API 할당량이 초과되지 않았는지")

---

#### 일반 URL에서 오디오 보내기

- 이 예시는 [`Kubernetes Podcast`](https://kubernetespodcast.com/) 에피소드의 오디오

In [ ]:
# 멀티모달 프롬프트에 사용할 오디오 URL
import requests
import pathlib
from google import genai

client = genai.Client()

# Download file
response = requests.get(
    "https://arxiv.org/html/1706.03762v7")
pathlib.Path('1706.03762v7.html').write_text(response.text)

my_file2 = client.files.upload(file='1706.03762v7.html')

try:
    # 이미지와 텍스트를 포함한 멀티모달 프롬프트 전송
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents = ["Summarize the document.",
                    my_file2]
    )

    # 생성된 텍스트 출력
    print(response.text)

except requests.exceptions.RequestException as e:
    print(f"이미지를 다운로드하는 중 오류가 발생했습니다: {e}")
except FileNotFoundError:
    print("지정된 로컬 이미지 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

<small>

* `pdf` 문서 (읽기 전용)이라 실패 → `HTML`로 도전 

* 셀 출력 (원문: contents = "Summarize the document.")(30.3s)

    ```markdown
    This document introduces the **Transformer**, a novel neural network architecture for sequence transduction models, such as machine translation. Unlike previous dominant models that rely on complex recurrent (RNNs) or convolutional neural networks, the Transformer **exclusively uses attention mechanisms**, completely discarding recurrence and convolutions.

    Key aspects and contributions of the Transformer:

    1.  **Architecture:** It maintains an encoder-decoder structure. Both the encoder and decoder are composed of stacked identical layers, primarily featuring **Multi-Head Self-Attention** and position-wise fully connected feed-forward networks.
        *   **Self-Attention** allows the model to weigh the importance of different parts of the input sequence when processing each element, enabling it to capture global dependencies without being limited by distance.
        *   **Multi-Head Attention** allows the model to attend to information from different representation subspaces in parallel, providing a richer understanding of relationships.
        *   **Positional Encodings** are added to input embeddings to incorporate information about the order and position of tokens in the sequence, a crucial addition since the model lacks intrinsic sequential processing.

    2.  **Advantages over RNNs/CNNs:**
        *   **Parallelization:** The attention-only architecture allows for significantly more parallel computation within training examples, reducing training time.
        *   **Shorter Path Lengths:** It reduces the path length between any two positions in the input or output sequence to a constant, which is theorized to make learning long-range dependencies easier.
        *   **Computational Efficiency:** It is often faster than recurrent layers when the sequence length is shorter than the representation dimensionality, common in many language tasks.

    3.  **Performance:**
        *   On the **WMT 2014 English-to-German translation task**, the Transformer achieved a new state-of-the-art BLEU score of 28.4, surpassing previous best results (including ensembles) by over 2 BLEU.
        *   For the **WMT 2014 English-to-French translation task**, it set a new single-model state-of-the-art BLEU score of 41.8, while requiring a significantly lower training cost (3.5 days on eight GPUs) compared to existing top models.
        *   The model also demonstrated good generalization by performing successfully on **English constituency parsing**.

    In summary, "Attention Is All You Need" introduces a revolutionary architecture that leverages attention mechanisms as its sole building block, leading to substantial improvements in training speed and translation quality, setting a new standard for sequence transduction tasks.
    ```

---

* 셀 출력 (한국어 버전: contents = "이 문서를 한국어로 요약해주세요")(33.5s)

    ```markdown
    이 문서는 "Attention Is All You Need"라는 제목의 논문을 한국어로 요약한 것입니다. 이 논문은 기존의 복잡한 순환 신경망(RNN)이나 합성곱 신경망(CNN) 기반의 인코더-디코더 모델과 달리, 오직 어텐션 메커니즘에만 기반한 새로운 신경망 구조인 **트랜스포머(Transformer)**를 제안합니다.

    **주요 내용:**

    1.  **배경 및 문제점:**
        *   기존 시퀀스 변환 모델(언어 모델링, 기계 번역 등)은 주로 RNN, 특히 LSTM이나 GRU에 기반하며, 인코더-디코더 구조에 어텐션 메커니즘이 추가된 형태입니다.
        *   RNN은 순차적인 계산 방식 때문에 훈련 중 병렬화가 어렵고, 긴 시퀀스에서 장거리 의존성(long-range dependency)을 학습하는 데 어려움이 있습니다.
        *   CNN 기반 모델(ByteNet, ConvS2S)은 병렬화가 가능하지만, 장거리 의존성을 모델링하려면 더 많은 레이어가 필요하여 경로 길이가 길어지는 한계가 있습니다.

    2.  **모델 아키텍처 (Transformer):**
        *   트랜스포머는 RNN이나 CNN 없이 **전적으로 어텐션 메커니즘**에 의존합니다.
        *   **인코더-디코더 구조:** 기존 모델과 유사하게 인코더와 디코더 스택으로 구성됩니다.
            *   **인코더:** N=6개의 동일한 레이어로 구성됩니다. 각 레이어는 Multi-Head Self-Attention 서브 레이어와 Position-wise Feed-Forward Network 서브 레이어로 이루어집니다.
            *   **디코더:** N=6개의 동일한 레이어로 구성됩니다. 인코더 레이어와 동일한 두 서브 레이어 외에, 인코더의 출력을 대상으로 하는 세 번째 Multi-Head Attention 레이어를 포함합니다. 디코더의 Self-Attention은 미래의 정보를 참조하지 않도록 마스킹됩니다.
            *   모든 서브 레이어에는 잔차 연결(Residual Connection)과 레이어 정규화(Layer Normalization)가 적용됩니다.
        *   **어텐션 메커니즘:**
            *   **Scaled Dot-Product Attention:** 쿼리(Query), 키(Key), 값(Value) 벡터를 입력받아, 쿼리와 모든 키의 내적을 계산하고, 이를 <img src="https://render.githubusercontent.com/render/math?math=%5Csqrt%7Bd_k%7D" alt="sqrt{d_k}">로 스케일링한 후 소프트맥스를 적용하여 값에 대한 가중치를 얻습니다.
            *   **Multi-Head Attention:** 단일 어텐션 함수 대신, 쿼리, 키, 값을 여러 개의 다른 학습된 선형 투영을 통해 작은 차원의 여러 "헤드"로 분할한 다음, 각 헤드에서 병렬로 어텐션 함수를 수행합니다. 이를 통해 모델이 여러 위치에서 다양한 표현 부분 공간의 정보에 동시에 집중할 수 있습니다. (논문에서는 h=8개의 헤드 사용)
            *   **모델 내 어텐션 활용:**
                *   **인코더-디코더 어텐션:** 디코더가 인코더의 모든 입력 시퀀스 위치를 참조할 수 있도록 합니다.
                *   **인코더 셀프-어텐션:** 인코더 내에서 각 위치가 이전 레이어의 모든 위치를 참조할 수 있도록 합니다.
                *   **디코더 셀프-어텐션:** 디코더 내에서 각 위치가 자기 자신을 포함하여 이전까지 생성된 모든 위치를 참조할 수 있도록 마스킹됩니다.
        *   **위치 인코딩(Positional Encoding):** 순환이나 합성곱이 없으므로, 시퀀스의 순서 정보를 모델에 주입하기 위해 입력 임베딩에 "위치 인코딩"을 더합니다. 이는 다양한 주파수의 사인 및 코사인 함수를 사용하여 고정된 방식으로 계산됩니다.

    3.  **셀프-어텐션의 장점:**
        *   **계산 효율성:** 각 레이어당 계산 복잡도가 RNN 레이어에 비해 효율적입니다 (특히 시퀀스 길이 <img src="https://render.githubusercontent.com/render/math?math=n">이 표현 차원 <img src="https://render.githubusercontent.com/render/math?math=d">보다 작을 때).
        *   **병렬화:** 순차적 계산이 O(1)에 불과하여 훈련 시 높은 병렬화를 가능하게 합니다.
        *   **장거리 의존성 학습:** 네트워크 내에서 입력 및 출력 위치 간의 최대 경로 길이가 상수(O(1))로 매우 짧아, 장거리 의존성 학습에 매우 유리합니다.
        *   **해석 가능성:** 어텐션 헤드들이 문장의 구문적, 의미적 구조와 관련된 다양한 역할을 학습함을 발견했습니다.

    4.  **훈련 및 결과:**
        *   **훈련 데이터:** WMT 2014 영어-독일어 (약 450만 문장 쌍) 및 영어-프랑스어 (약 3,600만 문장 쌍) 데이터셋.
        *   **훈련 비용:** 8개의 NVIDIA P100 GPU를 사용하여 기본 모델은 12시간, 대형 모델은 3.5일 훈련.
        *   **최적화:** Adam 옵티마이저를 사용하며, 학습률은 웜업(warmup) 스텝 이후 역제곱근에 비례하여 감소합니다.
        *   **정규화:** 잔차 드롭아웃(Residual Dropout)과 레이블 스무딩(Label Smoothing)을 적용했습니다.
        *   **성능:**
            *   **기계 번역:** WMT 2014 영어-독일어 번역 태스크에서 28.4 BLEU를 달성하여 기존 최고 성능 모델(앙상블 포함)보다 2 BLEU 이상 뛰어난 **최고 성능(State-of-the-Art, SOTA)**을 기록했습니다. 영어-프랑스어 번역에서도 41.8 BLEU로 단일 모델 최고 성능을 달성했으며, 훈련 비용은 기존 SOTA 모델의 일부에 불과했습니다.
            *   **영어 구성 요소 구문 분석:** 특정 태스크에 대한 튜닝 없이도 기존의 RNN 기반 모델보다 우수한 성능을 보여주며 다른 태스크로의 우수한 일반화 능력을 입증했습니다.

    **결론:**

    트랜스포머는 어텐션에 전적으로 기반한 최초의 시퀀스 변환 모델로서, 기존 순환 및 합성곱 모델보다 훨씬 빠른 훈련 속도를 제공하면서도 기계 번역 등 다양한 태스크에서 뛰어난 품질과 새로운 SOTA 성능을 달성했습니다. 저자들은 어텐션 기반 모델의 미래에 대한 기대를 표하며, 텍스트 외 다른 모달리티(이미지, 오디오, 비디오)로의 확장과 비순차적 생성 연구를 계획하고 있습니다. 모델의 코드는 텐서플로우(TensorFlow) 깃허브 저장소에 공개되어 있습니다.
    ```

---

#### `YouTube` `URL`에서 동영상 보내기

- 이 예시는 [Google — 25 Years in Search: The Most Searched](https://www.youtube.com/watch?v=3KtWfp0UopM) 유튜브 동영상

In [ ]:
# 멀티모달 프롬프트에 사용할 유튜브 주소 = "https://www.youtube.com/watch?v=3KtWfp0UopM"
from google import genai

client = genai.Client()

# URL 정보를 딕셔너리로 직접 전달
video_content = {
    "uri": "https://www.youtube.com/watch?v=3KtWfp0UopM",
    "mime_type": "video/mp4"
}

try:
    # 이미지와 텍스트를 포함한 멀티모달 프롬프트 전송
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents = ["At what point in the video is Harry Potter shown?",
                    video_content],
    )

    # 생성된 텍스트 출력
    print(response.text)

except requests.exceptions.RequestException as e:
    print(f"이미지를 다운로드하는 중 오류가 발생했습니다: {e}")
except FileNotFoundError:
    print("지정된 로컬 이미지 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

<small>

* 셀 출력 (변수 조절 X)(21.3s)

    ```markdown
    Harry Potter is not explicitly shown in the video. However, two characters from the Harry Potter franchise are featured:

    *   **Professor Snape** appears from **0:57 to 0:59**.
    *   **Hagrid** appears from **0:59 to 1:02**.
    ```

---

* 셀 출력 (한국어로 묻기)(content: "동영상에서 해리 포터가 언제 나오나요?")(13.2s)

    ```markdown
    동영상에서 해리 포터와 관련된 장면은 **0:56초부터 1:02초** 사이에 나옵니다.

    이때 'the most searched cast'라는 자막과 함께 스네이프 교수와 해그리드의 모습이 등장합니다.
    ```

---

#### 멀티모달 라이브 API

* `멀티모달 라이브 API`는 Gemini와 음성 및 영상으로 양방향 소통을 가능하게 하여, 대기 시간이 짧은(`low-latency`) 음성 및 영상 상호 작용을 지원 
  * 이 `API`를 사용하면 자연스럽고 사람과 같은 음성 대화 경험을 최종 사용자에게 제공하고, 음성 명령을 사용하여 모델의 응답을 중단할 수도 있음. 
  * 이 모델은 `텍스트`, `오디오`, `영상` 입력을 처리하고, `텍스트`와 `오디오` 출력을 제공

* `멀티모달 라이브 API`는 [WebSockets](https://developer.mozilla.org/en-US/docs/Web/API/WebSockets_API) 기반
  * `멀티모달 라이브 API`의 더 많은 예시는 다음 [문서](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-live) 또는 [노트북](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/multimodal-live-api/intro_multimodal_live_api_genai_sdk.ipynb)을 참조

In [ ]:
# 터미널에 사전 설치 : pip install pydantic

# pydantic = 데이터 유효성 검사 및 설정 관리를 위한 파이썬 라이브러리 = BaseModel 사용 위해 필요
from google import genai
from google.genai import types
from pydantic import BaseModel

client = genai.Client()

class Recipe(BaseModel):
    name: str
    description: str
    ingredients: list[str]


response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="List a few popular cookie recipes and their ingredients.",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=Recipe,
    ),
)

print(response,'\n')
print('-'*50, '\n')

# 파싱된 Recipe 객체를 변수에 저장
recipe_data = response.parsed

# 깔끔하게 출력
print(f"--- 레시피 정보 ---")
print(f"이름: {recipe_data.name}")
print(f"설명: {recipe_data.description}")
print(f"재료: ")
for ingredient in recipe_data.ingredients:
    print(f"- {ingredient}")
print(f"------------------")

<small>

* 셀 출력 (변수 조절 X)(1.3s)

    ```markdown
    sdk_http_response=HttpResponse(
    headers=<dict len=11>
    ) candidates=[Candidate(
    content=Content(
        parts=[
        Part(
            text="""{
    "name": "Chocolate Chip Cookies",
    "description": "A classic and beloved cookie, soft and chewy in the center with slightly crisp edges, loaded with chocolate chips.",
    "ingredients": [
        "All-purpose flour",
        "Baking soda",
        "Salt",
        "Unsalted butter",
        "Granulated sugar",
        "Brown sugar",
        "Eggs",
        "Vanilla extract",
        "Chocolate chips"
    ]
    }"""
        ),
        ],
        role='model'
    ),
    finish_reason=<FinishReason.STOP: 'STOP'>,
    index=0
    )] create_time=None response_id=None model_version='gemini-2.5-flash-lite' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=107,
    prompt_token_count=11,
    prompt_tokens_details=[
        ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=11
        ),
    ],
    total_token_count=118
    ) automatic_function_calling_history=[] parsed=Recipe(name='Chocolate Chip Cookies', description='A classic and beloved cookie, soft and chewy in the center with slightly crisp edges, loaded with chocolate chips.', ingredients=['All-purpose flour', 'Baking soda', 'Salt', 'Unsalted butter', 'Granulated sugar', 'Brown sugar', 'Eggs', 'Vanilla extract', 'Chocolate chips']) 

    -------------------------------------------------- 

    --- 레시피 정보 ---
    이름: Chocolate Chip Cookies
    설명: A classic and beloved cookie, soft and chewy in the center with slightly crisp edges, loaded with chocolate chips.
    재료: 
    - All-purpose flour
    - Baking soda
    - Salt
    - Unsalted butter
    - Granulated sugar
    - Brown sugar
    - Eggs
    - Vanilla extract
    - Chocolate chips
    ------------------
    ```

In [ ]:
# 파싱된 Recipe 객체를 변수에 저장
recipe_data = response.parsed

# 깔끔하게 출력
print(f"--- 레시피 정보 ---")
print(f"이름: {recipe_data.name}")
print(f"설명: {recipe_data.description}")
print(f"재료: ")
for ingredient in recipe_data.ingredients:
    print(f"- {ingredient}")
print(f"------------------")

<small>

* 셀 출력 (변수 조절 X)(0.0s)

    ```markdown
    --- 레시피 정보 ---
    이름: Chocolate Chip Cookies
    설명: A classic and beloved cookie, soft and chewy in the center with slightly crisp edges, loaded with chocolate chips.
    재료: 
    - All-purpose flour
    - Baking soda
    - Salt
    - Unsalted butter
    - Granulated sugar
    - Brown sugar
    - Eggs
    - Vanilla extract
    - Chocolate chips
    ------------------
    ```

---

### 생성된 출력 제어

* [출력 제어(Controlled generation)](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output)는 응답 스키마를 정의
  * `모델 출력의 구조`, `필드 이름` 및 `각 필드의 예상 데이터 유형`을 `지정`하는 기능입니다.
  * 응답 스키마는 `config`의 `response_schema` 매개변수에 지정 → 모델 출력은 해당 스키마를 엄격하게 따름

* 스키마 형태
  * [Pydantic](https://docs.pydantic.dev/) 
  * [JSON](https://www.json.org/json-en.html) 문자열로 제공

* 모델의 응답 :`response_mime_type`에 설정된 값에 따름
  *  `JSON` 
  *  [Enum](https://docs.python.org/3/library/enum.html)

In [ ]:
# 새 SDK는 pydantic 클래스를 사용하여 스키마를 제공
# genai.types.Schema 또는 이에 상응하는 dict을 전달할 수 있음
# 가능하면 SDK는 반환된 JSON을 파싱하고 결과를 response.parsed로 반환
# pydantic 클래스를 스키마로 제공한 경우 SDK는 해당 JSON을 클래스의 인스턴스로 변환

from google import genai
from pydantic import BaseModel

client = genai.Client()

class CountryInfo(BaseModel):
    name: str
    population: int
    capital: str
    continent: str
    major_cities: list[str]
    gdp: int
    official_language: str
    total_area_sq_mi: int

response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "rating": {"type": "INTEGER"},
                "flavor": {"type": "STRING"},
                "sentiment": {
                    "type": "STRING",
                    "enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"],
                },
                "explanation": {"type": "STRING"},
            },
            "required": ["rating", "flavor", "sentiment", "explanation"],
        },
    },
}


prompt = """
    Analyze the following product reviews, output the sentiment classification, and give an explanation.

    - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
    - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt,
    config={
        'response_mime_type': 'application/json',
        'response_schema': response_schema,
    },
)

print(response.parsed)

<small>

* 셀 출력 (변수 조절 X)(1.7s)

    ```markdown
    [[{'rating': 4, 'flavor': 'Strawberry Cheesecake', 'sentiment': 'POSITIVE', 'explanation': "The reviewer expresses strong positive sentiment, calling it the 'best ice cream ever'."}, {'rating': 1, 'flavor': 'Mango Tango', 'sentiment': 'NEGATIVE', 'explanation': "Despite stating it's 'quite good', the reviewer finds it 'too sweet', resulting in a negative sentiment reflected in the very low rating."}]]
    ````

In [10]:
# 중첩된 리스트 안에 딕셔너리가 들어가있는 결과값 → 반복문으로 꺼내기

# response.parsed는 중첩 리스트이므로 첫 번째 요소를 가져오기
parsed_reviews = response.parsed[0]

print("--- 분석 결과 ---")
for review in parsed_reviews:
    print(f"** 평점: {review['rating']}")
    print(f"** 맛: {review['flavor']}")
    print(f"** 감정: {review['sentiment']}")
    print(f"** 설명: {review['explanation']}")
    print("---") # 각 리뷰를 구분하는 줄

--- 분석 결과 ---
** 평점: 4
** 맛: Strawberry Cheesecake
** 감정: POSITIVE
** 설명: The reviewer expresses strong positive sentiment, calling it the 'best ice cream ever'.
---
** 평점: 1
** 맛: Mango Tango
** 감정: NEGATIVE
** 설명: Despite stating it's 'quite good', the reviewer finds it 'too sweet', resulting in a negative sentiment reflected in the very low rating.
---


---

### 토큰 개수 세기 및 계산

* `count_tokens()` 메서드를 사용하면 Gemini API로 요청을 보내기 전에 입력 토큰 수를 계산할 수 있음
  * 더 자세한 정보는 [토큰 나열 및 개수 세기](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/list-token) 문서를 참조

* **[`SDK 이전 코드`](https://ai.google.dev/gemini-api/docs/migrate?hl=ko&_gl=1*10gkk0*_up*MQ..*_ga*MTk4MzUwNjQyNy4xNzU1NDE1MzE2*_ga_P1DBVKWT6V*czE3NTU0MTcxNDEkbzIkZzAkdDE3NTU0MTcxNDEkajYwJGwwJGgxNTcwMTgyOTg5#json-response) 우선 참조**

#### 토큰 집계

In [ ]:
from google import genai

client = genai.Client()

response = client.models.count_tokens(
    model='gemini-2.0-flash',
    contents="What's the highest mountain in Africa?",
)

print(response)

<small>

* 셀 출력 (변수 조절 X)(0.4s)

    ```markdown
    sdk_http_response=HttpResponse(
    headers=<dict len=11>
    ) total_tokens=10 cached_content_token_count=None
    ```

---

  * 제공해주신 응답은 **총 입력 토큰 수가 10개**라는 것을 의미

    * **`total_tokens=10`**: 모델에 보낸 총 입력 토큰의 개수 = 이는 텍스트, 이미지, 동영상 등 모든 입력의 토큰을 합산한 값
    * **`sdk_http_response`**: `SDK`가 `HTTP`를 통해 서버와 통신하며 받은 `응답의 세부 정보`
    * **`cached_content_token_count=None`**: 캐시된 콘텐츠의 토큰 수가 없다는 의미 = 보통 동일한 프롬프트를 다시 보낼 때 나타날 수 있음

    * 내가 모델에 보낸 `프롬프트(텍스트, 코드, 이미지 등)`가 **총 10개의 토큰으로 구성되어 있으며, 이 토큰 수를 기반으로 비용이 청구될 것**이라는 것을 알 수 있음

  * `count_tokens()` 메서드 사용 →  `total_tokens` 값을 `API 호출 전`에 `미리 확인` 가능

#### 토큰 계산

* `compute_tokens()` 메서드는 `API 호출` 대신 `로컬 토크나이저를 실행` 
* 이 메서드는 `token_ids`(토큰 ID)와 `tokens`(실제 토큰)와 같은 더 자세한 토큰 정보를 제공

* 실행 코드
  
    ```python
    response = client.models.compute_tokens(
        model=MODEL_ID,
        contents="What's the longest word in the English language?",
    )

    print(response)
    ```

<br>

* 결과
* 
    ```markdown
    sdk_http_response=HttpResponse(
    headers=<dict len=9>
    ) tokens_info=[TokensInfo(
    role='user',
    token_ids=[
        1841,
        235303,
        235256,
        573,
        32514,
        <... 6 more items ...>,
    ],
    tokens=[
        b'What',
        b"'",
        b's',
        b' the',
        b' longest',
        <... 6 more items ...>,
    ]
    )]
    ```

<br>


* **참고: 이 메서드는 Vertex AI에서만 지원**

---

### **Grounding 검색**

* [그라운딩(Grounding)](https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview) = 현실의 `데이터`를 `Gemini 모델`과 `연결`

* 모델의 응답을 `Google 검색 결과에` `연결(grounding)` 
  * 모델은 훈련 데이터 범위를 넘어선 최신의 정확하고 관련성 있는 정보를 런타임에 접근하여 활용
  * `Google 검색`을 활용한 그라운딩을 통해 모델 응답의 정확성과 최신성을 향상
  * `Gemini 2.0부터`는 `Google 검색이 도구(tool)`로 `제공` = `모델`이 `Google 검색`을 언제 사용할지 `스스로 결정`

#### **Google 검색**

* `tools` 
  * 키워드 인수에 `GoogleSearch`를 포함한 `Tool`을 추가
  * `Gemini`가 프롬프트에 대해 **먼저 `Google 검색`을 수행** → 웹 검색 결과를 기반으로 답변을 구성

* [`동적 검색(Dynamic Retrieval)`](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-with-google-search#dynamic-retrieval)
  * 모델 응답에 그라운딩을 사용하는 시점에 대한 임계값을 설정
  * 프롬프트가 Google 검색 기반의 답변을 요구하지 않고, 지원되는 모델이 그라운딩 없이도 **자체 지식으로 답변을 제공할 수 있을 때 유용**
  * 이를 통해 `지연 시간(latency)`, 품질 및 비용을 보다 효과적으로 관리 가능

In [ ]:
from google import genai
from google.genai import types
from IPython.display import display, Markdown, HTML

client = genai.Client()

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents="When is the next total solar eclipse in the United States?",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                google_search=types.GoogleSearch()
            )
        ]
    )
)

display(Markdown(response.text))

print(response.candidates[0].grounding_metadata)

HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

<small>

* 셀 출력 (변수 조절 X)(2.3s)

    ```markdown
    The next total solar eclipse visible in the United States will occur on August 23, 2044. However, it will only be visible in Montana, North Dakota, and South Dakota.

    Another total solar eclipse will occur on August 12, 2045, and it will be visible across the entire continental U.S., spanning from California to Florida.
    ```

    ```js
    grounding_chunks=[GroundingChunk(
    web=GroundingChunkWeb(
        title='cbsnews.com',
        uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFYEigGPHQTNcRbBnS0_8SsxRYabW-eaMWgzqieLf7yPhQmU0TdQd8L_jJY4OsB-OC3Vga1XYipCyQ7FNIzf48aSz1gXtG46YUmB96FuxIuQQH51Ho99wGgKStNJ-q6nu0Y63el81CCmsUYeFoH-bi6YwbBu9Jmj576mA=='
    )
    ), GroundingChunk(
    web=GroundingChunkWeb(
        title='space.com',
        uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEltgDkd4qGBaYbZdaTlLdlHZJF-xmQydRD9rbAuMUD9yJqXrul76BrQBr8E6a7zdrWsTzDq0i7lDbrAPxAlwhSoYX69m1-CqtCS-Lu4ZMZ8cgK48_ugclPUYt5isFgWYj_pPINgOyeX6CZjEZwgtaP'
    )
    )] grounding_supports=[GroundingSupport(
    confidence_scores=[
        0.91026646,
        0.019670822,
    ],
    grounding_chunk_indices=[
        0,
        1,
    ],
    segment=Segment(
        end_index=87,
        text='The next total solar eclipse visible in the United States will occur on August 23, 2044'
    )
    ), GroundingSupport(
    confidence_scores=[
        0.8805385,
        0.032336947,
    ],
    grounding_chunk_indices=[
        1,
        0,
    ],
    segment=Segment(
        end_index=164,
        start_index=89,
        text='However, it will only be visible in Montana, North Dakota, and South Dakota'
    )
    ), GroundingSupport(
    confidence_scores=[
        0.94996846,
        0.0059847105,
    ],
    grounding_chunk_indices=[
        1,
        0,
    ],
    segment=Segment(
        end_index=320,
        start_index=167,
        text='Another total solar eclipse will occur on August 12, 2045, and it will be visible across the entire continental U.S., spanning from California to Florida'
    )
    )] retrieval_metadata=RetrievalMetadata() retrieval_queries=None search_entry_point=SearchEntryPoint(
    rendered_content="""<style>
    .container {
    align-items: center;
    border-radius: 8px;
    display: flex;
    font-family: Google Sans, Roboto, sans-serif;
    font-size: 14px;
    line-height: 20px;
    padding: 8px 12px;
    }
    .chip {
    display: inline-block;
    border: solid 1px;
    border-radius: 16px;
    min-width: 14px;
    padding: 5px 16px;
    text-align: center;
    user-select: none;
    margin: 0 8px;
    -webkit-tap-highlight-color: transparent;
    }
    .carousel {
    overflow: auto;
    scrollbar-width: none;
    white-space: nowrap;
    margin-right: -12px;
    }
    .headline {
    display: flex;
    margin-right: 4px;
    }
    .gradient-container {
    position: relative;
    }
    .gradient {
    position: absolute;
    transform: translate(3px, -9px);
    height: 36px;
    width: 9px;
    }
    @media (prefers-color-scheme: light) {
    .container {
        background-color: #fafafa;
        box-shadow: 0 0 0 1px #0000000f;
    }
    .headline-label {
        color: #1f1f1f;
    }
    .chip {
        background-color: #ffffff;
        border-color: #d2d2d2;
        color: #5e5e5e;
        text-decoration: none;
    }
    .chip:hover {
        background-color: #f2f2f2;
    }
    .chip:focus {
        background-color: #f2f2f2;
    }
    .chip:active {
        background-color: #d8d8d8;
        border-color: #b6b6b6;
    }
    .logo-dark {
        display: none;
    }
    .gradient {
        background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
    }
    }
    @media (prefers-color-scheme: dark) {
    .container {
        background-color: #1f1f1f;
        box-shadow: 0 0 0 1px #ffffff26;
    }
    .headline-label {
        color: #fff;
    }
    .chip {
        background-color: #2c2c2c;
        border-color: #3c4043;
        color: #fff;
        text-decoration: none;
    }
    .chip:hover {
        background-color: #353536;
    }
    .chip:focus {
        background-color: #353536;
    }
    .chip:active {
        background-color: #464849;
        border-color: #53575b;
    }
    .logo-light {
        display: none;
    }
    .gradient {
        background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
    }
    }
    </style>
    <div class="container">
    <div class="headline">
        <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
        <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
        <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
        <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
        <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
        </svg>
        <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
        <circle cx="24" cy="23" fill="#FFF" r="22"/>
        <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
        <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
        <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
        <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
        </svg>
        <div class="gradient-container"><div class="gradient"></div></div>
    </div>
    <div class="carousel">
        <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_UdYTzCMOInx6C-adQE3DSCrqpAMY9dVOZJjQKAXtLapXBrbe1oenOITWOecwdI3UfaFFjcfCQR4GSvrmylyXSQwH96zlPq2YLtTFbxFNSoTIw3rw3TGXhLmdVfzYleUSeIJARMviuvsGb8y2092k1PUcocY1wuf6PE-FYpb6QLJKKyRkzTiAtL2UmoTzrCQnVTgQNqpUu9mtUzfvNmQ6CHTR1ximZ5g2D-Wiow==">next total solar eclipse United States date</a>
    </div>
    </div>
    """
    ) web_search_queries=['next total solar eclipse United States date']
    ```

    ```css
    <style>
    .container {
    align-items: center;
    border-radius: 8px;
    display: flex;
    font-family: Google Sans, Roboto, sans-serif;
    font-size: 14px;
    line-height: 20px;
    padding: 8px 12px;
    }
    .chip {
    display: inline-block;
    border: solid 1px;
    border-radius: 16px;
    min-width: 14px;
    padding: 5px 16px;
    text-align: center;
    user-select: none;
    margin: 0 8px;
    -webkit-tap-highlight-color: transparent;
    }
    .carousel {
    overflow: auto;
    scrollbar-width: none;
    white-space: nowrap;
    margin-right: -12px;
    }
    .headline {
    display: flex;
    margin-right: 4px;
    }
    .gradient-container {
    position: relative;
    }
    .gradient {
    position: absolute;
    transform: translate(3px, -9px);
    height: 36px;
    width: 9px;
    }
    @media (prefers-color-scheme: light) {
    .container {
        background-color: #fafafa;
        box-shadow: 0 0 0 1px #0000000f;
    }
    .headline-label {
        color: #1f1f1f;
    }
    .chip {
        background-color: #ffffff;
        border-color: #d2d2d2;
        color: #5e5e5e;
        text-decoration: none;
    }
    .chip:hover {
        background-color: #f2f2f2;
    }
    .chip:focus {
        background-color: #f2f2f2;
    }
    .chip:active {
        background-color: #d8d8d8;
        border-color: #b6b6b6;
    }
    .logo-dark {
        display: none;
    }
    .gradient {
        background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
    }
    }
    @media (prefers-color-scheme: dark) {
    .container {
        background-color: #1f1f1f;
        box-shadow: 0 0 0 1px #ffffff26;
    }
    .headline-label {
        color: #fff;
    }
    .chip {
        background-color: #2c2c2c;
        border-color: #3c4043;
        color: #fff;
        text-decoration: none;
    }
    .chip:hover {
        background-color: #353536;
    }
    .chip:focus {
        background-color: #353536;
    }
    .chip:active {
        background-color: #464849;
        border-color: #53575b;
    }
    .logo-light {
        display: none;
    }
    .gradient {
        background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
    }
    }
    </style>
    <div class="container">
    <div class="headline">
        <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
        <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
        <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
        <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
        <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
        </svg>
        <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
        <circle cx="24" cy="23" fill="#FFF" r="22"/>
        <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
        <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
        <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
        <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
        </svg>
        <div class="gradient-container"><div class="gradient"></div></div>
    </div>
    <div class="carousel">
        <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_UdYTzCMOInx6C-adQE3DSCrqpAMY9dVOZJjQKAXtLapXBrbe1oenOITWOecwdI3UfaFFjcfCQR4GSvrmylyXSQwH96zlPq2YLtTFbxFNSoTIw3rw3TGXhLmdVfzYleUSeIJARMviuvsGb8y2092k1PUcocY1wuf6PE-FYpb6QLJKKyRkzTiAtL2UmoTzrCQnVTgQNqpUu9mtUzfvNmQ6CHTR1ximZ5g2D-Wiow==">next total solar eclipse United States date</a>
    </div>
    </div>
    ```

<small>

* 위 css로 생긴 버튼을 눌렀을 경우 구글 크롬 검색창으로 바로 이동 
  * ![결과 이미지](../resources/result.png)

#### Vertex AI Search

* [Vertex AI Search 데이터 스토어](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es)를 사용하여 Gemini를 당신의 고유한 맞춤형 데이터와 연결 가능

* 자세한 내용은 [Vertex AI Search 시작하기 가이드](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search)를 참고

---

### 함수 호출(Function calling)

* `Gemini`의 [함수 호출(Function Calling)](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) 기능
  * 코드 내 함수의 설명을 작성한 다음, 해당 설명을 언어 모델에 요청으로 전달 가능
  * `자동 함수 호출`을 위해 파이썬 함수를 제출 → `Gemini`가 함수를 `실행`하고 그 `결과`를 `자연어로 생성`하여 반환

* [OpenAPI Specification](https://www.openapis.org/)을 제출 → 설명에 맞는 함수의 이름과 호출 시 사용할 인수를 응답으로 받을 수도 있음

In [ ]:
# 함수 호출

from google import genai
from google.genai import types

client = genai.Client()

def get_current_weather(location: str) -> str:
    """Get the current whether in a given location.

    Args:
        location: required, The city and state, e.g. San Franciso, CA
        unit: celsius or fahrenheit
    """
    print(f'Called with: {location=}')
    return "23C"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(
        tools=[get_current_weather],
        automatic_function_calling={'disable': True},
    ),
)

function_call = response.candidates[0].content.parts[0].function_call

print(function_call)

<small>

* 셀 출력 (변수 조절 X)(0.9s)

    ```markdown
    id=None args={'location': 'Boston, MA'} name='get_current_weather'
    ```

In [ ]:
# 자동 함수 호출

from google import genai
from google.genai import types

client = genai.Client()

def get_current_weather(city: str) -> str:
    return "23C"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(
        tools=[get_current_weather]
    ),
)


print(response.text)                        # It is 23C in Boston. (0.0s)


print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  avg_logprobs=-0.023808644711971284,
  content=Content(
    parts=[
      Part(
        text="""It is 23C in Boston.
"""
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>
)] create_time=None response_id=None model_version='gemini-2.0-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=10,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=10
    ),
  ],
  prompt_token_count=33,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=33
    ),
  ],
  total_token_count=43
) automatic_function_calling_history=[UserContent(
  parts=[
    Part(
      text='What is the weather like in Boston?'
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
       

<small>

* 셀 출력 (변수 조절 X)(1.8s)

    ```markdown
    sdk_http_response=HttpResponse(
    headers=<dict len=11>
    ) candidates=[Candidate(
    avg_logprobs=-0.023808644711971284,
    content=Content(
        parts=[
        Part(
            text="""It is 23C in Boston.
    """
        ),
        ],
        role='model'
    ),
    finish_reason=<FinishReason.STOP: 'STOP'>
    )] create_time=None response_id=None model_version='gemini-2.0-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=10,
    candidates_tokens_details=[
        ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=10
        ),
    ],
    prompt_token_count=33,
    prompt_tokens_details=[
        ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=33
        ),
    ],
    total_token_count=43
    ) automatic_function_calling_history=[UserContent(
    parts=[
        Part(
        text='What is the weather like in Boston?'
        ),
    ],
    role='user'
    ), Content(
    parts=[
        Part(
        function_call=FunctionCall(
            args={
            'city': 'Boston'
            },
            name='get_current_weather'
        )
        ),
    ],
    role='model'
    ), Content(
    parts=[
        Part(
        function_response=FunctionResponse(
            name='get_current_weather',
            response={
            'result': '23C'
            }
        )
        ),
    ],
    role='user'
    )] parsed=None
    ```

---

*  `generate_content()` 메서드가 반환하는 **응답 객체의 모든 값(속성)을 그대로 출력한 것**
   * 파이썬 객체의 `문자열 표현(string representation)`
   * `Gemini API`와 주고받은 통신에 대한 `모든 디테일`과 모델의 `최종 응답`

---

* 응답 객체 분석

* 최종 응답 (`candidates` 섹션)

  * **`candidates`**: 모델이 생성한 응답 후보들을 담고 있는 리스트
  * **`content`**: 모델의 실제 응답 내용
      * **`text="""It is 23C in Boston."""`**
        * 모델이 최종적으로 생성하여 사용자에게 보여주는 자연어 텍스트
        * 이 부분이 `response.text`를 통해 접근하는 값

* 토큰 및 비용 정보 (`usage_metadata` 섹션)

  * **`usage_metadata`**: API 사용량 및 비용 관련 정보를 제공
      * **`prompt_token_count=33`**: 당신이 모델에 보낸 입력 프롬프트의 토큰 수
      * **`candidates_token_count=10`**: 모델이 생성한 응답 텍스트의 토큰 수
      * **`total_token_count=43`** = **총 사용된 토큰 수(입력 + 출력)** = 이 숫자를 기준으로 **비용 청구**

* 함수 호출 기록 (`automatic_function_calling_history` 섹션)

  * **`automatic_function_calling_history`**: 모델이 함수 호출을 수행한 과정에 대한 상세 기록
      * **`role='user'`**: 사용자가 `"What is the weather like in Boston?"`이라고 질문
      * **`role='model'`**: 모델이 내부적으로 `get_current_weather` 함수를 호출하기로 결정하고 `city` 인자로 `'Boston'`을 지정
      * **`role='user'`**: (모델이 호출한 함수가 실행된 후) 그 결과값인 `{'result': '23C'}`가 다시 모델로 입력됨

* **결론적으로, 이 출력은 Gemini가 당신의 질문을 이해하고, 날씨 정보를 가져오는 함수를 호출하여, 그 결과를 바탕으로 최종 답변 텍스트를 생성하는 모든 과정을 상세히 보여주는 '보고서'와 같음.**

In [ ]:
# 구글 클라우드 실습 함수 예시로 시도

def get_current_weather2(location: str) -> str:
    """Example method. Returns the current weather.

    Args:
        location: The city and state, e.g. San Francisco, CA
    """
    weather_map: dict[str, str] = {
        "Boston, MA": "snowing",
        "San Francisco, CA": "foggy",
        "Seattle, WA": "raining",
        "Austin, TX": "hot",
        "Chicago, IL": "windy",
    }
    return weather_map.get(location, "unknown")


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the weather like in Austin, TX?",
    config=types.GenerateContentConfig(
        tools=[get_current_weather2],
        temperature=0,
    ),
)

print(response.text)

<small>

* 셀 출력_1 (temperature=0)(2.3s)

    ```markdown
    Warning: there are non-text parts in the response: ['thought_signature'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
    Could you provide the state for Austin?
    ```

    ---

    ```markdown
    경고: 응답에 텍스트가 아닌 부분이 있습니다. ['thought_signature']에서 텍스트 부분만 연결하여 반환합니다. 전체 모델 응답을 보려면 `candidates.content.parts` 접근자를 확인하세요.

    Austin의 주(state)를 알려주실 수 있나요?
    ```

<small>

* 셀 출력_2 (tempemrature=0)(2.2s)

* 주 입력 요청에 따라 contents 수정 → contents="What is the weather like in Austin, TX?",

    ```markdown

    The weather in Austin, TX is hot.

    ```

---

#### OpenAPI 상세화 (수동 함수 호출)

In [ ]:
from google import genai

client = genai.Client()

# --- client 객체를 사용하여 "헬로우" 호출해보기---
# 1. 사용할 모델 지정 (gemini-2.5-flash-lite)

# 2. '헬로우' 프롬프트로 콘텐츠 생성 요청
try:
    response = client.models.generate_content(
        model='gemini-2.5-flash-lite',                     # 사용할 모델 지정 (gemini-2.5-flash-lite)
        contents='hello'                                   # 요청하신 프롬프트 '헬로우'
    )

    # 응답 텍스트 출력
    print("\n--- 모델 응답 텍스트 ---")
    print(response.text)

    # 응답 전체 내용(JSON 형식) 출력 (디버깅이나 상세 정보 확인용)
    print("\n--- 모델 응답 전체 JSON ---")
    print(response.model_dump_json(
        exclude_none=True, indent=4))

except Exception as e:
    print(f"\n모델 호출 중 오류가 발생했습니다: {e}")
    print("다음 사항들을 확인해주세요:")
    print("1. 인터넷 연결 상태")
    print("2. GEMINI_API_KEY 환경 변수가 올바르고 유효한지")
    print("3. Google Cloud 프로젝트에서 Gemini API가 활성화되어 있는지")
    print("4. API 할당량이 초과되지 않았는지")

In [18]:
from openai import OpenAI

client = OpenAI(
    api_key="GEMINI_API_KEY",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


# 모든 함수를 딕셔너리로 정의하기
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Chicago, IL",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    },
    # get_destination 함수를 딕셔너리 형태로 추가
    {
        "type": "function",
        "function": {
            "name": "get_destination",
            "description": "Get the destination that the user wants to go to",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination": {
                        "type": "string",
                        "description": "Destination that the user wants to go to",
                    },
                },
                "required": ["destination"],
            },
        }
    }
]

MODEL_ID='gemini-2.5-flash'

response = client.chat.completions.create(                              # chat.completions.create 사용
    model=MODEL_ID,
    messages=[
        {"role": "user", "content": "I'd like to travel to Paris."},
    ],
    tools=tools, # 딕셔너리 리스트로 정의된 tools 사용
)

print(response.choices[0].message.tool_calls[0].function)

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}]

In [19]:
from google import genai
from google.genai import Tool, FunctionDeclaration

# 1. 새 SDK의 클라이언트 객체를 생성
client = genai.Client()

# 2. 함수를 FunctionDeclaration 클래스로 정의
get_destination = FunctionDeclaration(
    name="get_destination",
    description="Get the destination that the user wants to go to",
    parameters={
        "type": "OBJECT",
        "properties": {
            "destination": {
                "type": "STRING",
                "description": "Destination that the user wants to go to",
            },
        },
        "required": ["destination"],
    },
)

# 3. Tool 객체에 함수 선언을 담기
destination_tool = Tool(
    function_declarations=[get_destination],
)

# 4. client 객체를 통해 모델 호출
# config에 tools를 전달하여 함수 호출을 활성화
response = client.models.generate_content(
    model='gemini-2.0-flash', # 모델 ID
    contents="I'd like to travel to Paris.",
    config=genai.types.GenerateContentConfig(
        tools=[destination_tool],
    ),
)

# 5. 함수 호출 결과 출력
print(response.function_calls[0])

ImportError: cannot import name 'Tool' from 'google.genai' (/Users/jay/.pyenv/versions/gcs_env/lib/python3.13/site-packages/google/genai/__init__.py)

In [ ]:
from google import genai

# 1. 새 SDK의 클라이언트 객체를 생성
client = genai.Client()

# 2. 함수 명세를 딕셔너리로 직접 정의
# FunctionDeclaration과 Tool 클래스를 사용하지 않음
destination_tool_spec = {
    "function_declarations": [
        {
            "name": "get_destination",
            "description": "Get the destination that the user wants to go to",
            "parameters": {
                "type": "OBJECT",
                "properties": {
                    "destination": {
                        "type": "STRING",
                        "description": "Destination that the user wants to go to",
                    },
                },
                "required": ["destination"],
            },
        },
    ],
}



# 3. client 객체를 통해 모델 호출
# tools 인수에 딕셔너리 명세를 직접 전달
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents="I'd like to travel to Paris.",
        config=types.GenerateContentConfig(
        tools=[destination_tool_spec],  
    )
)

# 4. 함수 호출 결과 출력
print(response.function_calls[0],'/n')
print(response)
print(destination_tool_spec)


<small>

* 셀 출력

  ```markdown

  id=None args={'destination': 'Paris'} name='get_destination'
  
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ) candidates=[Candidate(
    avg_logprobs=3.701401874423027e-06,
    content=Content(
      parts=[
        Part(
          function_call=FunctionCall(
            args={
              'destination': 'Paris'
            },
            name='get_destination'
          )
        ),
      ],
      role='model'
    ),
    finish_reason=<FinishReason.STOP: 'STOP'>
  )] create_time=None response_id=None model_version='gemini-2.0-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=5,
    candidates_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=5
      ),
    ],
    prompt_token_count=34,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=34
      ),
    ],
    total_token_count=39
  ) automatic_function_calling_history=[] parsed=None
  {'function_declarations': [{'name': 'get_destination', 'description': 'Get the destination that the user wants to go to', 'parameters': {'type': 'OBJECT', 'properties': {'destination': {'type': 'STRING', 'description': 'Destination that the user wants to go to'}}, 'required': ['destination']}}]}
  ```

#### next

In [32]:
from google import genai
from google.genai.types import Function, UserMessage
import json

client = genai.Client()                 

# 함수 선언
get_weather = Function(
    name="get_weather",
    description="전달된 위치의 날씨를 가져옵니다",
    parameters={
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "도시 이름, 예: Seoul"},
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
        },
        "required": ["location"]
    }
)

# 함수 호출 요청
response = client.chat.completions.create(
    model="gemini-2.5-flash",  
    messages=[UserMessage(content="서울의 날씨 알려줘")],
    functions=[get_weather],
    function_call="auto"
)

# 함수 호출 응답 확인
if response.choices[0].message.function_call:
    name = response.choices[0].message.function_call.name
    arguments = response.choices[0].message.function_call.arguments
    args_dict = json.loads(arguments)
    print("함수 호출 이름:", name)
    print("인자:", args_dict)


ImportError: cannot import name 'Function' from 'google.genai.types' (/Users/jay/.pyenv/versions/gcs_env/lib/python3.13/site-packages/google/genai/types.py)

In [33]:
from google import genai
from google.genai import types

client = genai.Client()

# 함수 정의
def get_current_weather(location: str, unit: str = "celsius") -> str:
    """현재 날씨를 가져오는 함수"""
    print(f"Called with: location={location}, unit={unit}")
    return "23°C and sunny"

# Gemini 모델에 함수 전달
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What's the weather in Seoul?",
    config=types.GenerateContentConfig(
        tools=[get_current_weather],  # Python 함수 직접 등록
        automatic_function_calling={"disable": False},  # 자동 호출 허용
    ),
)

# 모델이 함수 호출을 했는지 확인
part = response.candidates[0].content.parts[0]
if hasattr(part, "function_call"):
    print("함수 호출:", part.function_call.name)
    print("인자:", part.function_call.args)


Called with: location=Seoul, unit=celsius


AttributeError: 'NoneType' object has no attribute 'name'

In [34]:
from google import genai
from google.genai import types

client = genai.Client()

def get_current_weather(location: str, unit: str = "celsius") -> str:
    print(f"Called with: location={location}, unit={unit}")
    return "23°C and sunny"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What's the weather in Seoul?",
    config=types.GenerateContentConfig(
        tools=[get_current_weather],
        automatic_function_calling={"disable": False},  # 자동 함수 호출 허용
    ),
)

# 응답 확인
for candidate in response.candidates:
    for part in candidate.content.parts:
        if part.function_call:
            print("함수 호출됨 ✅")
            print("함수 이름:", part.function_call.name)
            print("인자:", part.function_call.args)
        elif part.text:
            print("모델 텍스트 응답:", part.text)


Called with: location=Seoul, unit=celsius
모델 텍스트 응답: It's 23°C and sunny in Seoul.



<small>

* 셀 출력 (1.4s)

    ```markdown
    Called with: location=Seoul, unit=celsius
    모델 텍스트 응답: It's 23°C and sunny in Seoul.
    ```
